In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import requestsimport matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import json
from pathlib import Path
from pprint import pprint
import hvplot.pandas
import requests
import warnings
warnings.filterwarnings("ignore")

In [ ]:
wind_turbine = "turbinedata.csv"

# Read the turbine data
wind = pd.read_csv(wind_turbine)

# Display the data table for preview
wind.head(5)

In [ ]:
wind_turbine_clean_df = wind[['case_id', 't_state', 'p_name', 'p_year', 'p_tnum', 't_manu', 't_model', 't_cap', 
                                        't_hh', 't_rd', 't_ttlh', 't_conf_atr', 't_conf_loc', 'xlong', 'ylat']]
wind_turbine_clean_df = wind_turbine_clean_df.rename(columns={'case_id':'Turbine ID', 't_state':'State','p_name':'Project Name',
                                                             'p_tnum':'Turbines in Project','t_manu':'Manufacturer',
                                                              't_model':'Model Number','t_cap':'Capacity',
                                                              'p_year':'Year','t_hh':'Height','t_rd':'Rotor Diameter',
                                                              't_ttlh':'Total Height','xlong':'Longitude','ylat':'Latitude'
                                                              })
wind_turbine_clean_df

In [ ]:
# Checking the number of turbines.
turbine_count = len(pd.unique(wind['case_id']))
turbine_count

In [ ]:
# Checking the number of manufactures.
manu_count = len(pd.unique(wind['t_manu']))
manu_count

In [ ]:
# Chekcing the number of models
model_count = len(pd.unique(wind['t_model']))
model_count

In [ ]:
new_wind = wind[['case_id','p_name','t_state','p_year','t_cap','t_manu','t_model','t_hh','xlong','ylat']]
new_wind

sp_turbine = new_wind.loc[new_wind["case_id"] == 3053439]
  
sp_turbine

In [ ]:
turbine_manu = wind["t_manu"].value_counts().head(15)
turbine_manu

turbine_manu.plot(kind="bar")

plt.title ('Top 15 Manufactures')
plt.xlabel('Manufacture')
plt.ylabel('# of Models')

plt.show()

In [ ]:
turbine_model = wind["t_model"].value_counts().head(20)
turbine_model

turbine_model.plot(kind="bar")

plt.title ('Total Models')
plt.xlabel('Models')
plt.ylabel('# of Models')

plt.show()

In [ ]:
turbine_cap = wind_turbine_clean_df.groupby(['Capacity'])
count_turbine = turbine_cap['Capacity'].count()
count_turbine

count_chart = count_turbine.plot(kind='bar', figsize=(20,20))

count_chart.set_xlabel("Capacity")
count_chart.set_ylabel("Number of Turbines")

plt.show()
plt.tight_layout()

In [ ]:
manu_pie = wind_turbine_clean_df['Manufacturer'].value_counts()
print(manu_pie)

labels = ["GE Wind", "Vestas", "Siemens","Gamesa", "Mitsubishi"]

sizes = [32300,17161,5166,3037,2373]

colors = ["red", "green","blue","orange"]

explode = (0.1, 0, 0, 0)

plt.pie(sizes, labels=labels, colors=colors,
        autopct="%0.2f%%", shadow=True)

plt.show()

In [ ]:
def autopct(pct): 
    return ('%.2f' % pct) if pct > 2 else ''


manu_data = wind_turbine_clean_df["Manufacturer"].value_counts()
manu_data.plot.pie(autopct= autopct)

plt.title("Manufacturers")
plt.show()

In [ ]:
plt.style.use('ggplot')
dic = {'GE Wind': 46.63,
       'Vestas': 24.77,
       'Siemens': 7.46,
       'Gamesa': 4.38,
       'Mitsubishi': 3.43,
       'Siemens Gamesa Renewable': 2.950598782,
       'Nordex': 2.505121613,
       'Suzlon': 1.809407268,
       'Acciona': 1.042196587,
       'REpower': 1.09,
       'NEG Micon': 0.610468713,
       'Enron': 0.544472096 ,
       'Bonus': 0.419353508,
       'Clipper': 0.38360534 ,
       'Nordtank': 0.318983652,
       'Micon': 0.290110132,
       'Goldwind': 0.268111259,
       'Northern Power Systems': 0.134743094}
# group together all elements in the dictionary whose value is less than 1
# name this group 'All the rest'
import itertools
newdic={}
for key, group in itertools.groupby(dic, lambda k: 'Other' if (dic[k]<2) else k):
     newdic[key] = sum([dic[k] for k in list(group)])
labels = newdic.keys()
sizes = newdic.values()
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=0)
ax.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# Remove any null values in the Capacity column
wind_capacity_cleaned_df = wind_turbine_clean_df.dropna()
wind_capacity_cleaned_df

In [ ]:
# Find turbines by state and plot the results
state_values = wind_capacity_cleaned_df.groupby(['State']).size()
state_values

state_values.plot(kind= 'bar', xlabel = 'State', ylabel ='Turbines', title = 'Turbines by State',  figsize=(10,4))

In [ ]:
eighties_turbines = wind_capacity_cleaned_df.loc[((wind_turbine_clean_df['Year'] >= 1980) &
                                                  (wind_turbine_clean_df['Year'] <= 1989))]                                                
nineties_turbines = wind_capacity_cleaned_df.loc[((wind_turbine_clean_df['Year'] >= 1990) &
                                                  (wind_turbine_clean_df['Year'] < 2000))] 
two_thousand_turbines = wind_capacity_cleaned_df.loc[((wind_turbine_clean_df['Year'] >= 2000) &
                                                  (wind_turbine_clean_df['Year'] < 2010))]
two_thousand_ten_turbines = wind_capacity_cleaned_df.loc[((wind_turbine_clean_df['Year'] >= 2010) &
                                                  (wind_turbine_clean_df['Year'] < 2020))]
two_thousand_twenty_turbines = wind_capacity_cleaned_df.loc[((wind_turbine_clean_df['Year'] >= 2021) &
                                                  (wind_turbine_clean_df['Year'] < 2024))]

In [ ]:
# Get the average capacity for each decade
eighties_capacity_avg = eighties_turbines['Capacity'].mean()
nineties_capacity_avg = nineties_turbines['Capacity'].mean()
two_thousand_capacity_avg = two_thousand_turbines['Capacity'].mean()
two_thousand_ten_capacity_avg = two_thousand_ten_turbines['Capacity'].mean()
two_thousand_twenty_capacity_avg = two_thousand_twenty_turbines['Capacity'].mean() 

In [ ]:
eighties_turbines['Capacity'].plot.hist(title = '1980s Turbines', color = 'green' )
avg_text = 'Avg Capacity= ' + str(round(eighties_capacity_avg,2)) + " kW"
plt.annotate(avg_text,(450, 600),fontsize=10,color="black")

In [ ]:
nineties_turbines['Capacity'].plot.hist(title = '1990s Turbines', color = 'purple')
avg_text = 'Avg Capacity= ' + str(round(nineties_capacity_avg,2)) + " kW"
plt.annotate(avg_text,(1000,600),fontsize=10,color="black")

In [ ]:
two_thousand_turbines['Capacity'].plot.hist(title = '2000s Turbines', color = 'blue')
avg_text = 'Avg Capacity= ' + str(round(two_thousand_capacity_avg,2)) + " kW"
plt.annotate(avg_text,(1570,6000),fontsize=10,color="black")

In [ ]:
two_thousand_ten_turbines['Capacity'].plot.hist(title = '2010s Turbines', color = 'red')
avg_text = 'Avg Capacity= ' + str(round(two_thousand_ten_capacity_avg,2)) + " kW"
plt.annotate(avg_text,(2800,12500),fontsize=10,color="black")

In [ ]:
two_thousand_twenty_turbines['Capacity'].plot.hist(title = '2020s Turbines', color = 'yellow')
avg_text = 'Avg Capacity= ' + str(round(two_thousand_twenty_capacity_avg,2)) + " kW"
plt.annotate(avg_text,(3500,3000),fontsize=10,color="black")

In [ ]:
two_thousand_ten_turbines['Capacity'].plot.hist(color = 'red')
two_thousand_turbines['Capacity'].plot.hist(color = 'blue')
eighties_turbines['Capacity'].plot.hist(color = 'green' )
nineties_turbines['Capacity'].plot.hist(color = 'purple')
two_thousand_twenty_turbines['Capacity'].plot.hist(color = 'yellow')
plt.title('Turbines 1980 to Present')
plt.show()

In [ ]:
# Build scatter plot for rotor diameter vs. capacity
capacity = wind_capacity_cleaned_df['Capacity']
rotor_diameter = wind_capacity_cleaned_df['Rotor Diameter']
plt.scatter(rotor_diameter, capacity, edgecolors='black', c='blue')

# Plot the linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(rotor_diameter, capacity)
regress_values = rotor_diameter * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq,(25,4500),fontsize=15,color="black")
plt.plot(rotor_diameter,regress_values,"r-")
print(f"The r-value is: {rvalue}")

# Incorporate the other graph properties
plt.xlabel('Rotor Diameter')
plt.ylabel('Turbine Capacity')
plt.title('Rotor Diameter vs. Turbine Capacity')
plt.grid()

# Show plot
plt.show()

In [ ]:
# Build scatter plot for rotor height vs. capacity
capacity = wind_capacity_cleaned_df['Capacity']
rotor_diameter = wind_capacity_cleaned_df['Height']
plt.scatter(rotor_diameter, capacity, edgecolors='black', c='blue')

# Plot the linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(rotor_diameter, capacity)
regress_values = rotor_diameter * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq,(25,4500),fontsize=15,color="black")
plt.plot(rotor_diameter,regress_values,"r-")
print(f"The r-value is: {rvalue}")

# Incorporate the other graph properties
plt.xlabel('Height')
plt.ylabel('Turbine Capacity')
plt.title('Height vs. Turbine Capacity')
plt.grid()

# Show plot
plt.show()

In [ ]:
x_axis = wind_all_cleaned_df['Year']
y_axis = wind_all_cleaned_df['Rotor Diameter']

plt.scatter(x_axis, y_axis)
plt.xlabel("Year")  # add X-axis label
plt.ylabel("Rotor Diameter (m)")  # add Y-axis label
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept, 2))
plt.plot(x_axis,regress_values,"r-")
plt.title("Rotor Diameter vs. Year") 
plt.show()

print(f"The trend from wind turbines across the United States indicates that average rotor diameter is increasing by {slope} meters each year.")


In [ ]:
gewind_turbines = wind_capacity_cleaned_df.loc[wind_capacity_cleaned_df['Manufacturer'] == 'GE Wind', "Capacity"] 
print(f"GE Wind Average Capacity: {gewind_turbines.mean()}")


vestas_turbines = wind_capacity_cleaned_df.loc[wind_capacity_cleaned_df['Manufacturer'] == 'Vestas', "Capacity"] 
print(f"Vestas Average Capacity: {vestas_turbines.mean()}")

siemens_turbines = wind_capacity_cleaned_df.loc[wind_capacity_cleaned_df['Manufacturer'] == 'Siemens', "Capacity"] 
print(f"Siemens Average Capacity: {siemens_turbines.mean()}")

gamesa_turbines = wind_capacity_cleaned_df.loc[wind_capacity_cleaned_df['Manufacturer'] == 'Gamesa', "Capacity"] 
print(f"Gamesa Average Capacity: {gamesa_turbines.mean()}")

mitsubishi_turbines = wind_capacity_cleaned_df.loc[wind_capacity_cleaned_df['Manufacturer'] == 'Mitsubishi', "Capacity"] 
print(f"Mitsubishi Average Capacity: {mitsubishi_turbines.mean()}")

In [ ]:
average_capacity = [1989, 2160, 2340, 1855, 1326]
lifespan = [2, 8, 70, 1.5, 25]
index = ['GE Wind', 'Vestas', 'Siemens',
         'Gamesa', 'Mitsubishi']

df = pd.DataFrame({'average capacity': average_capacity,

                   'lifespan': lifespan}, index=index)

ax = df.plot.bar(rot=0)

In [ ]:
base_url = "https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_state=eq.AZ"
response = requests.get(base_url)
data = response.json()

# Print to see data returned
# print(data)  

latitudes = []
longitudes = []

# Loop to extract latitudes and longitudes
for item in data:
    ylat = item['ylat']
    xlong = item['xlong']
    latitudes.append(ylat)
    longitudes.append(xlong)

# DataFrame with latitude and longitude information
az_wind_location_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

In [ ]:
az_wind_location_df

In [ ]:
# Configure the map plot_2
map_plot_az = az_wind_location_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "red",  
)

# Display the map plot
map_plot_az

In [ ]:
base_url = "https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_state=eq.CA"
response = requests.get(base_url)
data = response.json()

# Print to see data returned
# print(data)  

latitudes = []
longitudes = []

# Loop to extract latitudes and longitudes
for item in data:
    ylat = item['ylat']
    xlong = item['xlong']
    latitudes.append(ylat)
    longitudes.append(xlong)

# DataFrame with latitude and longitude information
ca_wind_location_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

In [ ]:
ca_wind_location_df

In [ ]:
map_plot_ca = ca_wind_location_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "red",  
)

# Display the map plot
map_plot_ca

In [ ]:
base_url = "https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_state=eq.TX"
response = requests.get(base_url)
data = response.json()

# Print to see data returned
# print(data)  

latitudes = []
longitudes = []

# Loop to extract latitudes and longitudes
for item in data:
    ylat = item['ylat']
    xlong = item['xlong']
    latitudes.append(ylat)
    longitudes.append(xlong)

# DataFrame with latitude and longitude information
tx_wind_location_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

In [ ]:
tx_wind_location_df

In [ ]:
map_plot_tx = tx_wind_location_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "red",  
)

# Display the map plot
map_plot_tx

In [ ]:
base_url = "https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_state=eq.OK"
response = requests.get(base_url)
data = response.json()

# Print to see data returned
# print(data)  

latitudes = []
longitudes = []

# Loop to extract latitudes and longitudes
for item in data:
    ylat = item['ylat']
    xlong = item['xlong']
    latitudes.append(ylat)
    longitudes.append(xlong)

# DataFrame with latitude and longitude information
ok_wind_location_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

In [ ]:
ok_wind_location_df

In [ ]:
map_plot_ok = ok_wind_location_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "red",  
)

# Display the map plot
map_plot_ok

In [ ]:
base_url = "https://eersc.usgs.gov/api/uswtdb/v1/turbines?&t_state=eq.IA"
response = requests.get(base_url)
data = response.json()

# Print to see data returned
# print(data)  

latitudes = []
longitudes = []

# Loop to extract latitudes and longitudes
for item in data:
    ylat = item['ylat']
    xlong = item['xlong']
    latitudes.append(ylat)
    longitudes.append(xlong)

# DataFrame with latitude and longitude information
ia_wind_location_df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

In [ ]:
ia_wind_location_df

In [ ]:
map_plot_ia = ia_wind_location_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.01,
    color = "red",  
)

# Display the map plot
map_plot_ia